In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [13]:
# Tickers
tickers = ["NVDA", "NFLX", "GME", "BB", "NOK", "AMZN", "TSLA"]
etfs = ["SPY", "IWM", "DIA"]


class DataDownloader:
    """
    DataDownloader is a class designed to fetch historical stock and ETF data.
    
    Attributes:
    - tickers: A list of stock tickers.
    - etfs: A list of ETF tickers.
    
    Methods:
    - download(period): Downloads historical data for the given period. Combines tickers and etfs.
    - quarterly(): Downloads data for the past 3 months.
    - annual(): Downloads data for the past year.
    - decade(): Downloads data for the past 10 years.
    
    """
    def __init__(self, tickers, etfs):
        self.tickers = tickers
        self.etfs = etfs

    def download(self, period='1y'):
        return yf.download(self.tickers + self.etfs, period=period)

    def quarterly(self):
        return self.download(period='3mo')

    def annual(self):
        return self.download(period='1y')
    
    def decade(self):
        return self.download(period='10y')
    

"""
After defining the class, an instance of the DataDownloader is created using the provided tickers and ETFs. 
The historical data for the tickers and ETFs is then fetched for the three time periods and stored in 
`quarterly_data`, `annual_data`, and `decade_data`.
"""
downloader = DataDownloader(tickers, etfs)
# Usage Guide:
quarterly_data = downloader.quarterly()
annual_data = downloader.annual()
decade_data = downloader.decade()


[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


In [14]:
# Trading days = count of rows per period
TRADING_DAYS_QUARTER = quarterly_data['Adj Close']['NVDA'].count()
TRADING_DAYS_ANNUAL = annual_data['Adj Close']['NVDA'].count()
TRADING_DAYS_DECADE = decade_data['Adj Close']['NVDA'].count()

## Utility functions for financial calculations:

- calculate_weights(prices): Computes the weightage of each stock based on its price relative to the total price of all stocks.
- calculate_annualized_volatility(ticker_data): Calculates the annualized volatility for a given stock using its daily returns.
- calculate_beta(stock, etf, returns): Computes the beta value of a stock relative to a specified ETF.


In [15]:
def calculate_weights(prices):
    total_price = prices.sum()
    return (prices / total_price) * 100

def calculate_annualized_volatility(ticker_data):
    daily_return = ticker_data.pct_change().dropna()
    std_dev = daily_return.std()
    return std_dev * math.sqrt(TRADING_DAYS_QUARTER) * 100

def calculate_beta(stock, etf, returns):
    covariance = returns[stock].cov(returns[etf])
    variance = returns[etf].var()
    return str(round(covariance / variance,2))

## Creating columns

In [16]:
# 1.Create table
part_one = pd.DataFrame(index=tickers)

# 2.Calculate weights
calc_weight = calculate_weights(quarterly_data['Adj Close'][tickers].iloc[-1])
weight = calc_weight.round(2).astype(str) + "%"
part_one['Weight'] = weight

# 3.Calculate annualized volatility
calc_vol = calculate_annualized_volatility(quarterly_data['Adj Close'][tickers])
volatility = calc_vol.round(2).astype(str) + "%"
part_one['Annualized Volatility'] = volatility
                                           
# 4-6.Calculate beta
returns = annual_data['Adj Close'].pct_change().dropna()
for etf in etfs:
    part_one[f'Beta against {etf}'] = [calculate_beta(stock, etf, returns) for stock in tickers]

# 7-8.Calculate average and maximum weekly drawdown
highs = annual_data['High'][tickers].pct_change().dropna()
lows = annual_data['Low'][tickers].pct_change().dropna()
part_one['Average Weekly Drawdown'] = (((lows.mean() - highs.mean()) / highs.mean()) * 100).round(2)
part_one['Maximum Weekly Drawdown'] = (((lows.min() - highs.max()) / highs.max()) * 100).round(2)

# 9-10.Calculate total return and annualized total return
initial_values = decade_data['Adj Close'][tickers].dropna().iloc[0]
current_values = decade_data['Adj Close'][tickers].iloc[-1]
part_one['Total Return'] = (((current_values - initial_values) / initial_values) * 100).round(2).astype(str) + "%"
part_one['Annualized Total Return'] = ((((current_values / initial_values) ** (1/10)) - 1) * 100).round(2).astype(str) + "%"




# Part One Complete

In [17]:
part_one

,Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return
NVDA,36.33%,19.82%,2.15,1.3,1.88,0.34,-128.46,12310.87%,61.95%
NFLX,30.85%,17.32%,1.43,0.94,1.39,2.41,-176.71,727.52%,23.53%
GME,1.25%,21.53%,1.63,1.44,1.67,63.71,-142.06,72.51%,5.6%
BB,0.36%,34.18%,1.6,1.22,1.56,-42.44,-207.67,-43.08%,-5.48%
NOK,0.3%,12.53%,1.11,0.82,1.23,-10.33,-319.96,-26.79%,-3.07%
AMZN,10.5%,16.09%,1.54,0.99,1.34,6.12,-203.87,722.6%,23.46%
TSLA,20.41%,24.56%,1.89,1.36,1.59,27.15,-205.75,2077.6%,36.08%


In [18]:

"""
This code computes the daily percentage change for the adjusted close prices, 
removes any missing values, and calculates the daily total value of the portfolio.
"""

# Calculate the percentage change for the adjusted close and drop NaN values.
annual_data_complete = annual_data['Adj Close'].pct_change().dropna()
annual_data_complete['portfolio'] = annual_data_complete[tickers].sum(axis=1)



In [19]:

"""
This code block defines functions to calculate various financial metrics:
1. Portfolio correlation with each ETF.
2. Portfolio covariance with each ETF.
3. Tracking error for each ETF over a specified period.
4. Sharpe ratio for each ETF.
5. Annualized volatility spread between the portfolio and each ETF.

Additionally, it preprocesses the annual data by 
calculating percentage changes and computing the portfolio's total value for each day.
"""

# 1
def calculate_portfolio_correlation(data, etfs, portfolio):
    return [round(data[etf].corr(data[portfolio]), 2) for etf in etfs]

# 2
def calculate_portfolio_covariance(data, etfs, portfolio):
    return [round(data[[etf, portfolio]].cov().loc[etf, portfolio] * 100, 2) for etf in etfs]

# 3
def calculate_tracking_error(data, etfs, tickers, trading_days):
    errors = []
    for etf in etfs:
        data['total_pct_change_tickers'] = data['Adj Close'][tickers + [etf]].sum(axis=1)
        data['TE'] = data['total_pct_change_tickers'] - data['Adj Close'][etf]
        te = (data['TE'].std() * 100) * math.sqrt(trading_days)
        errors.append(round(te, 2))
    return errors

# 4
def calculate_sharpe_ratio(data, etfs, tickers):
    ratios = []
    for etf in etfs:
        data['total_pct_change_tickers'] = data[tickers + [etf]].sum(axis=1)
        daily_return_mean = data['total_pct_change_tickers'].mean()
        daily_return_stdev = data['total_pct_change_tickers'].std()
        ratio = (daily_return_mean / daily_return_stdev) * 100
        ratios.append(round(ratio, 2))
    return ratios

# 5
def calculate_ann_vol_spread(data, etfs, tickers, trading_days):
    spreads = []
    for etf in etfs:
        port_voli = (data[tickers].sum(axis=1).std()) * math.sqrt(trading_days)
        etf_voli = (data[etf].std()) * math.sqrt(trading_days)
        spreads.append(round((port_voli - etf_voli) * 100, 2))
    return spreads

## Create table

In [20]:
#0: Create table
part_two = pd.DataFrame(index=etfs)


#1-2: apply correlation & covariance
part_two['Correlation against ETF'] = calculate_portfolio_correlation(annual_data_complete, etfs, 'portfolio')
part_two['Covariance against ETF'] = calculate_portfolio_covariance(annual_data_complete, etfs, 'portfolio')


#3: apply Tracking erros
part_two['Tracking Errors'] = calculate_tracking_error(decade_data.copy(), etfs, tickers, TRADING_DAYS_DECADE)


#4: apply Sharpe Ratio
part_two['Sharpe Ratio'] = calculate_sharpe_ratio(annual_data_complete, etfs, tickers)


#5: apply AVS
part_two['Annualized Volatility Spread'] = calculate_ann_vol_spread(annual_data_complete, etfs, tickers, TRADING_DAYS_ANNUAL)

## Part two complete

In [21]:
part_two

,Correlation against ETF,Covariance against ETF,Tracking Errors,Sharpe Ratio,Annualized Volatility Spread
SPY,0.79,0.13,1996595.56,5.78,224.75
IWM,0.71,0.15,1996595.56,5.44,220.22
DIA,0.62,0.08,1996595.56,5.81,227.45
